## Análisis de Simulated Annealing para una fuerza de ventas de 10 nodos

Simulated Annhealing es un algoritmo que recibe los siguientes hiperárametros de entrada: 

+ Tmax- Temperatura máxima con la que inicia el algoritmo
+ Tmin- Tempertatura mínima a la que llega al equilibrio
+ steps- Número de iteraciones
+ updates- Número de updates. 

Los resultados obtenidos pueden verse afectados al variar los valores de tales hiperparámetros. Por otro lado, dependiendo del número de nodos del grafo, estos hiperparámetros también podrían afectar la ruta mínima encontrada por el algoritmo.    

El presente notebook considera la implementación de simulated annhealing para un grafo con 10 nodos. Como primero objetivo se variarán los hiperparámetros y se identificarán aquellos que den mejores resultados. Entendiéndose como mejores resultados; obtener la ruta con menor distancia. Adicionalmente, una vez seleccionados los mejores hiperparámetros, se correrá el algoritmo 100 veces con el fin de realizar un análisis sobre las rutas obtenidas. Particularmente se tiene interés en revisar variaciones en las rutas obtenidas en cada corrida.

Dentro del conjunto de datos que se tienen disponibles, existen varias fuerzas de venta que deben recorrer 10 nodos. Se decidió elegir la fuerza de venta **96298** perteneciente al estado de Nuevo León para llevar a cabo estas pruebas.

In [1]:
!pip install dynaconf
!pip install psycopg2-binary
!pip install simanneal

In [2]:
# Librerías

import pandas as pd
import sys
sys.path.append('../../')
%load_ext autoreload
%autoreload 2

from src import Utileria as ut
from src.models import particle_swarm as ps
from src.models import simulated_annealing as sa

### 1. Búsqueda de mejores hiperparámetros

**1.1 Definición de datos de entrada**
+ Grafo completo de los puntos que debe vistar la fuerza de ventas
+ Hiperámetros con los que correrá el algoritmo

In [3]:
# Se obtiene el dataframe que contiene el grafo de la fuerza de venta a evaluar:
str_Query = 'select id_origen, id_destino, distancia from trabajo.grafos where id_fza_ventas={};'

# En el query se especifica el id_fza_venta del cual se quiere obtener su grafo
df_Grafo = ut.get_data(str_Query.format(96298))
df_Grafo

,id_origen,id_destino,distancia
0,11078,1009790047,3.1011127017081943
1,11078,1009791566,0.5142577124821974
2,11078,1020088646,0.9248048280652852
3,11078,1020249367,0.47811555481864304
4,11078,1020253076,0.6722824338378225
5,11078,1020300220,0.7785165511359913
6,11078,1020328100,0.9287372777889837
7,11078,1020449326,0.3605529558796124
8,11078,1020451581,0.46907337618604666
9,1009790047,1009791566,3.609300132764086


In [4]:
# Se crea el diccionario de hiper-parámetros que se evaluarán

#Default parameters
#Tmax = 25000.0  # Max (starting) temperature
#Tmin = 2.5      # Min (ending) temperature
#steps = 50000   # Number of iterations
#updates = 100   # Number of updates (by default an update prints to stdout)

dict_Hiper_SA = {'tmax': {10000, 25000},
              'tmin': {1,2.5,5},
              'steps': {500,5000},
              'updates': {10,50,100}
              }



**1.2 Gridsearch**

Dentro de la clase Utileria fue definido un método llamado *GridSearch*, el cual recibe como parámetros el grafo de una fuerza de ventas fijo, un diccionario de parámetros, el algoritmo a evaluar y el número de iteraciones que se correrá por cada combinación de hiperámetros. Este método evalúa el algoritmo con todas las combinaciones que se pueden generar a partir del diccionario de parámetros. 

En este caso se considerarán 3 valores de *updates*, 2 valores del *steps*, 2 valores de *Tmax* y 2 de *Tmin* dando lugar a un total de 36 combinaciones. 

Cada combinación de hiperarámetros se correrá 100 veces y como resultado se obtendrá una tabla indicando los Hiperámetros utilizados, las distancias mínima y máxima obtenidas dentro de las 100 corridas; y el número de corridas en que se repitió tal distancia mínima.

In [5]:
%%time

# Se corre el GridSearch para el grafo y los hiperparámetros previamente definidos

df_Resultado = ut.GridSearch(df_Grafo, sa.SimulatedAnnealing, dict_Hiper_SA, 100)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          9.93    82.00%    32.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000         10.07    68.00%    32.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          9.09    74.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          8.93    80.00%    42.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000         11.00    90.00%    32.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000         10.41    82.00%    32.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000         10.39    90.00%    40.00%     0:00:00     0:00:00 Temperature 

CPU times: user 29min 22s, sys: 1min 2s, total: 30min 25s
Wall time: 29min 59s


In [6]:
# Se muestra el dataframe con los resultados obtenidos de la corrida del GridSearch
pd.options.display.max_colwidth = 100
df_Resultado

,HiperParámetros,Distancia mínima (km),Distancia máxima (km),Frec. rel. dist. min.
0,"{'tmax': 10000, 'tmin': 1.0, 'steps': 5000, 'updates': 100}",8.458,8.821,6/100
1,"{'tmax': 10000, 'tmin': 1.0, 'steps': 5000, 'updates': 10}",8.458,8.866,2/100
2,"{'tmax': 10000, 'tmin': 1.0, 'steps': 5000, 'updates': 50}",8.458,8.817,2/100
3,"{'tmax': 10000, 'tmin': 1.0, 'steps': 500, 'updates': 100}",8.576,9.277,1/100
4,"{'tmax': 10000, 'tmin': 1.0, 'steps': 500, 'updates': 10}",8.559,9.194,1/100
5,"{'tmax': 10000, 'tmin': 1.0, 'steps': 500, 'updates': 50}",8.572,9.205,1/100
6,"{'tmax': 10000, 'tmin': 2.5, 'steps': 5000, 'updates': 100}",8.458,8.840,1/100
7,"{'tmax': 10000, 'tmin': 2.5, 'steps': 5000, 'updates': 10}",8.458,8.814,1/100
8,"{'tmax': 10000, 'tmin': 2.5, 'steps': 5000, 'updates': 50}",8.458,8.855,1/100
9,"{'tmax': 10000, 'tmin': 2.5, 'steps': 500, 'updates': 100}",8.459,9.286,1/100


**1.3 Análisis y Resultados**

En primer lugar es importante mencionar las motivaciones de los hiperparámetros elegidos. 

+ Con respecto al número de iteraciones,se escogieron $100$ iteraciones para poder ver la variación entre iteraciones y la estabilidad del algoritmo. El número fue suficiente ya que se observa que en la mayoría de combinaciones de hiperparámetros la frecuencia de ruta mínima es bastante baja, siendo $6/100$ la mayor frecuencia. 

+ El resto de los parámetros, se eligieron cercanos a los valores por omisión que presenta el algoritmo, los cuales también se incluyeron.

Para poder interpretar los resultados mostrados en el dataframe anterior es conveniente recordar que se realizaron 36 pruebas, es decir, se tuvieron 36 combinaciones de hiperparámetros; y cada una de esas combinaciones se corrió 100 veces, dando un total de 360 corridas. En las 100 corridas de cada prueba se registró la distancia mínima obtenida, la distancia máxima y la frecuencia relativa de la distancia mínima. A continuación se enlistan observaciones importantes derivadas de estas pruebas:

+ La distancia mínima encontrado por el algoritmo se encuentra a lo largo del intervalo: $[8.458,8.567]km$, mientras que la máxima en: $[8.809,9.417]km$. De esta manera es evidente que la distancia entre el mínimo global y el máximo global es bastante pequeña(menos de 1 km) y aunado a que se presenta la misma distanica mínima($8.458$), en la mayoria de combinaciones de hiperparámetros; se puede decir que el algoritmo presenta muy poca variación en su ejecución. 
 
+ Por otro lado, aunque las distancias arrojadas por todas las combinaciones son muy similares, es cierto que las frecuencias relativas observadas a lo largo de las 100 iteraciones son bastante bajas. Comparando con la ejecución realizada para la fuerza de ventas con 6 nodos, esta frecuencia disminuye drasticamente, por lo que se puede concluir que un número mayor de nodos desestabiliza al algoritmo.

In [7]:
%%time

# Se corre el GridSearch para el grafo y los hiperparámetros previamente definidos
dict_Hiper_SA = {'tmax': {2}, 'tmin': {1}, 'steps': {1}, 'updates': {2}}

df_Resultado = ut.GridSearch(df_Grafo, sa.SimulatedAnnealing, dict_Hiper_SA, 100)
pd.options.display.max_colwidth = 100
df_Resultado

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000         11.20   100.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          9.54     0.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000         10.57   100.00%   100.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          9.89   100.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000         10.57   100.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000         11.31   100.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000         10.34   100.00%   100.00%     0:00:00     0:00:00 Temperature 

CPU times: user 37.5 s, sys: 544 ms, total: 38.1 s
Wall time: 37.3 s


,HiperParámetros,Distancia mínima (km),Distancia máxima (km),Frec. rel. dist. min.
0,"{'tmax': 2, 'tmin': 1, 'steps': 1, 'updates': 2}",9.1,11.545,1/100


### 2. GridSearch con los mejores parámetros

Se corre el algortimo con la combinación de hiperparámetros que arrojó mejores resultados en la parte anterior.

In [8]:
#{'tmax': 10000, 'tmin': 1.0, 'steps': 5000, 'updates': 100}	
rutas = pd.DataFrame(index=range(100),columns=['km', 'Ruta'])


dict_Hiper_SA = {'tmax': 10000, 'tmin': 1.0, 'steps': 5000, 'updates': 100}
              



In [9]:
rutas

,km,Ruta
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
95,NaN,NaN
96,NaN,NaN
97,NaN,NaN
98,NaN,NaN


In [10]:
for corrida in range(100):
    SA = sa.SimulatedAnnealing(df_Grafo,dict_Hiper_SA)
    SA.Ejecutar()
    
    min_distancia = round(SA.nbr_MejorCosto,3)
    mejor_ruta =  ut.convert(SA.lst_MejorCamino)
    
    rutas.km[corrida] = min_distancia
    rutas.Ruta[corrida] = mejor_ruta
    
    

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          9.67    82.00%    34.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000         10.90    86.00%    36.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          9.84    92.00%    42.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          9.90    88.00%    44.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000         11.14    86.00%    36.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000         10.97    84.00%    36.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000         10.97    88.00%    34.00%     0:00:00     0:00:00 Temperature 

In [11]:
# Primeras 7 rutas obtenidas
pd.options.display.max_colwidth = 100
rutas.head(7)

,km,Ruta
0,8.566,11078-1009790047-1020449326-1020249367-1020328100-1020088646-1020300220-1020253076-1009791566-10...
1,8.726,11078-1020253076-1020300220-1020088646-1020328100-1009791566-1020451581-1020249367-1020449326-10...
2,8.666,11078-1009790047-1020451581-1009791566-1020253076-1020328100-1020300220-1020088646-1020249367-10...
3,8.675,11078-1020449326-1020300220-1020088646-1020328100-1020253076-1020249367-1009791566-1020451581-10...
4,8.678,11078-1009790047-1020449326-1020328100-1020088646-1020300220-1020253076-1020249367-1009791566-10...
5,8.566,11078-1009791566-1020451581-1020328100-1020088646-1020300220-1020253076-1020249367-1020449326-10...
6,8.576,11078-1009790047-1020449326-1020249367-1020253076-1020328100-1020088646-1020300220-1009791566-10...


In [14]:
# Descripción de las rutas obtenidas
pd.options.display.max_colwidth = 100
results = rutas.groupby('Ruta').describe()
results

km  \
                                                                                                     count   
Ruta                                                                                                         
11078-1009790047-1009791566-1020451581-1020249367-1020300220-1020088646-1020328100-1020253076-102...   1.0   
11078-1009790047-1009791566-1020451581-1020253076-1020088646-1020300220-1020328100-1020249367-102...   1.0   
11078-1009790047-1009791566-1020451581-1020328100-1020088646-1020300220-1020253076-1020249367-102...   1.0   
11078-1009790047-1009791566-1020451581-1020328100-1020300220-1020088646-1020253076-1020249367-102...   1.0   
11078-1009790047-1020249367-1020253076-1020088646-1020300220-1020328100-1009791566-1020451581-102...   2.0   
...                                                                                                    ...   
11078-1020451581-1009791566-1020328100-1020088646-1020300220-1020253076-1020249367-1020449326-100...   3.0   
11078-1020451581-1009791566-1020328100-1020253076-1020088646-1020300220-1020249367-1020449326-100...   1.0   
11078-1020451581-1009791566-1020328100-1020253076-1020300220-1020088646-1020249367-1020449326-100...   1.0   
11078-1020451581-1009791566-1020449326-1020249367-1020253076-1020328100-1020088646-1020300220-100...   2.0   
11078-1020451581-1009791566-1020449326-1020300220-1020088646-1020328100-1020253076-1020249367-100...   1.0   

                                                                                                             \
                                                                                                     unique   
Ruta                                                                                                          
11078-1009790047-1009791566-1020451581-1020249367-1020300220-1020088646-1020328100-1020253076-102...    1.0   
11078-1009790047-1009791566-1020451581-1020253076-1020088646-1020300220-1020328100-1020249367-102...    1.0   
11078-1009790047-1009791566-1020451581-1020328100-1020088646-1020300220-1020253076-1020249367-102...    1.0   
11078-1009790047-1009791566-1020451581-1020328100-1020300220-1020088646-1020253076-1020249367-102...    1.0   
11078-1009790047-1020249367-1020253076-1020088646-1020300220-1020328100-1009791566-1020451581-102...    1.0   
...                                                                                                     ...   
11078-1020451581-1009791566-1020328100-1020088646-1020300220-1020253076-1020249367-1020449326-100...    1.0   
11078-1020451581-1009791566-1020328100-1020253076-1020088646-1020300220-1020249367-1020449326-100...    1.0   
11078-1020451581-1009791566-1020328100-1020253076-1020300220-1020088646-1020249367-1020449326-100...    1.0   
11078-1020451581-1009791566-1020449326-1020249367-1020253076-1020328100-1020088646-1020300220-100...    1.0   
11078-1020451581-1009791566-1020449326-1020300220-1020088646-1020328100-1020253076-1020249367-100...    1.0   

                                                                                                             \
                                                                                                        top   
Ruta                                                                                                          
11078-1009790047-1009791566-1020451581-1020249367-1020300220-1020088646-1020328100-1020253076-102...  8.756   
11078-1009790047-1009791566-1020451581-1020253076-1020088646-1020300220-1020328100-1020249367-102...  8.787   
11078-1009790047-1009791566-1020451581-1020328100-1020088646-1020300220-1020253076-1020249367-102...  8.573   
11078-1009790047-1009791566-1020451581-1020328100-1020300220-1020088646-1020253076-1020249367-102...  8.687   
11078-1009790047-1020249367-1020253076-1020088646-1020300220-1020328100-1009791566-1020451581-102...  8.705   
...                                                                                                     ...   
11078-102

Corriendo 100 veces el algoritmo con los mejores hiperparámetros obtenidos antes mencionados, se registran 76 rutas diferentes con diversas distancias mínimas, las cuales oscilan muy poco (al rededor de .3 km de diferencia). Se observan muy pocas repeticiones de ruta, siendo 4 repeticiones la máxima por ruta, y la ruta con la distancia mínima se observa únicamente en 5 ocasiones, lo que equivale al 5% del total de rutas arrojadas.
De esta forma podemos conlcuir que la variación de rutas es muy grande en cada iteración, pero las distancias mínimas entre cada ruta encontrada por el algoritmo son cortas.

### Conclusiones:
+ El hecho de aumentar el número de nodos a recorrer, ocasiona que el algortimo pierda cierta estabilidad, arrojando variaciones pequeñas en las rutas mńimas encontradas.
+ Aunque las distancias mínimas encontradas por el algoritmo son muy similares, las rutas halladas a lo largo de diferntes iteraciones del algoritmo, discrepan unas a otras en la mayoría de los casos.